In [2]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import association_rules, apriori

### Read Dataset

In [3]:
sales_reciepts = pd.read_csv ('dataset/201904 sales reciepts.csv')
sales_reciepts.head()

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N


In [4]:
product = pd.read_csv ('dataset/product.csv')
product.head()

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.40,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.40,$18.00,Y,N,N
2,3,Whole Bean/Teas,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,1 lb,11.80,$14.75,Y,N,N
3,4,Whole Bean/Teas,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,16.36,$20.45,Y,N,N
4,5,Whole Bean/Teas,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,12.00,$15.00,Y,N,N


### Data Wrangling

#### Merge Data

In [5]:
dataset = pd.merge(
    sales_reciepts[['transaction_id','transaction_date', 'sales_outlet_id', 'customer_id', 'product_id', 'quantity']],
    product[['product_id', 'product_category', 'product']],
    on ='product_id', 
    how = 'left' 
)
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
0,7,2019-04-01,3,558,52,1,Tea,Traditional Blend Chai Rg
1,11,2019-04-01,3,781,27,2,Coffee,Brazilian Lg
2,19,2019-04-01,3,788,46,2,Tea,Serenity Green Tea Rg
3,32,2019-04-01,3,683,23,2,Coffee,Our Old Time Diner Blend Rg
4,33,2019-04-01,3,99,34,1,Coffee,Jamaican Coffee River Sm


### Reduce Product Sizes

In [6]:
dataset[dataset['product'].str.contains('Dark chocolate')]['product'].unique()

array(['Dark chocolate Lg', 'Dark chocolate Rg', 'Dark chocolate'],
      dtype=object)

In [7]:
dataset['product'].nunique()

80

In [8]:
dataset['product']= dataset['product'].str.replace(' Rg','')
dataset['product']= dataset['product'].str.replace(' Sm','')
dataset['product']= dataset['product'].str.replace(' Lg','')

In [9]:
dataset['product'].nunique()

45

### Choose Product Subset

In [10]:
products_to_take = ['Cappuccino', 'Latte', 'Espresso shot',  \
                     'Dark chocolate','Sugar Free Vanilla syrup', 'Chocolate syrup',\
                    'Carmel syrup', 'Hazelnut syrup', 'Ginger Scone',  \
                    'Chocolate Croissant', 'Jumbo Savory Scone', 'Cranberry Scone', 'Hazelnut Biscotti',\
                    'Croissant', 'Almond Croissant', 'Oatmeal Scone', 'Chocolate Chip Biscotti',\
                    'Ginger Biscotti',\
                   ]

In [11]:
dataset = dataset[
    dataset['product'].isin(products_to_take)
]

In [12]:
dataset['product'].nunique()

18

In [13]:
dataset[
    ['product','product_category']
].drop_duplicates().reset_index(drop=True)

,product,product_category
0,Cappuccino,Coffee
1,Espresso shot,Coffee
2,Latte,Coffee
3,Dark chocolate,Drinking Chocolate
4,Oatmeal Scone,Bakery
5,Jumbo Savory Scone,Bakery
6,Chocolate Chip Biscotti,Bakery
7,Ginger Biscotti,Bakery
8,Chocolate Croissant,Bakery
9,Hazelnut Biscotti,Bakery


### Clean Transactions

In [14]:
dataset['transaction'] = dataset['transaction_id'].astype(str) + "_"+  dataset['customer_id'].astype(str)

In [15]:
num_of_items_for_each_transaction = dataset['transaction'].value_counts().reset_index()

num_of_items_for_each_transaction

,transaction,count
0,209_0,31
1,206_0,30
2,204_0,27
3,208_0,25
4,203_0,24
...,...,...
8381,135_523,1
8382,130_157,1
8383,121_465,1
8384,118_748,1


In [16]:
valid_transactions = num_of_items_for_each_transaction[
    (num_of_items_for_each_transaction['count']>1)
]['transaction'].tolist()

valid_transactions

['209_0',
 '206_0',
 '204_0',
 '208_0',
 '203_0',
 '207_0',
 '205_0',
 '151_0',
 '323_0',
 '301_0',
 '42_0',
 '9_0',
 '76_0',
 '202_0',
 '213_0',
 '307_0',
 '274_0',
 '149_0',
 '46_0',
 '51_0',
 '370_0',
 '242_0',
 '298_0',
 '195_0',
 '197_0',
 '475_0',
 '6_0',
 '329_0',
 '344_0',
 '317_0',
 '102_0',
 '59_0',
 '54_0',
 '325_0',
 '172_0',
 '255_0',
 '310_0',
 '118_0',
 '319_0',
 '223_0',
 '140_0',
 '132_0',
 '3_0',
 '97_0',
 '36_0',
 '216_0',
 '282_0',
 '314_0',
 '10_0',
 '135_0',
 '320_0',
 '146_0',
 '278_0',
 '313_0',
 '75_0',
 '211_0',
 '43_0',
 '234_0',
 '122_0',
 '144_0',
 '291_0',
 '171_0',
 '266_0',
 '253_0',
 '356_0',
 '251_0',
 '61_0',
 '352_0',
 '421_0',
 '334_0',
 '228_0',
 '284_0',
 '30_0',
 '315_0',
 '187_0',
 '345_0',
 '377_0',
 '38_0',
 '289_0',
 '239_0',
 '164_0',
 '268_0',
 '143_0',
 '152_0',
 '410_0',
 '295_0',
 '327_0',
 '311_0',
 '227_0',
 '170_0',
 '177_0',
 '260_0',
 '45_0',
 '85_0',
 '32_0',
 '359_0',
 '280_0',
 '235_0',
 '190_0',
 '306_0',
 '262_0',
 '264_0',
 '4

In [17]:
dataset = dataset[dataset['transaction'].isin(valid_transactions)]

In [18]:
dataset.shape

(10189, 9)